# 검색개수 기반 복합명사 추출

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

## 데이터

In [2]:
df = pd.read_csv('dataset.csv')
df.head()

,title,answer,mecab
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,소비자보호실태평가 결과 어디 확인,"소비자 보호 실태 평가 결과,확인"
1,소비자보호실태평가의 평가방법이 궁금합니다.,소비자보호실태평가 평가방법,"소비자 보호 실태 평가,평가 방법"
2,소비자보호실태평가는 어떤 제도인가요?,소비자보호실태평가 제도,"소비자 보호 실태 평가,제도"
3,금융감독원의 민원대상 금융회사가 궁금합니다.,금융감독원 민원대상 금융회사,"금융 감독원,민원 대상 금융 회사"
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,내 금융계좌 번 조회 사이트,"금융 계좌,조회,사이트"


## 명사-명사 시퀀스 검색 & 점수 부여

In [3]:
def naver_search(rawNoun):
    basic_url= "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query="
    headers = {'User-Agent' : '...'}
    
    # 파싱
    html = requests.get(basic_url+rawNoun, headers = headers).text
    soup = BeautifulSoup(html, "lxml")
    mainPack = str(soup.find('div', 'main_pack'))
    
    # 파워링크
    plAd = str(soup.find('div', "ad_section section _pl_section bg_type_v2"))
    # 비즈사이트
    bzAd = str(soup.find('div', "ad_section section _bz_section bg_type_v2"))
    # 파워콘텐츠
    bgAd = str(soup.find('div', "ad_section section bg_type_v2"))
    
    count = mainPack.count(rawNoun) - plAd.count(rawNoun) - bzAd.count(rawNoun) - bgAd.count(rawNoun)
    
    return count

In [4]:
def permute(s):
    result = [[s]]
    for i in range(1, len(s)):
        first = [s[:i]]
        rest = s[i:]
        for p in permute(rest):
            result.append(first + p)
    return result

In [5]:
def scoring(nouns, weight, verbose=0):
    result = []
    for noun in nouns:
        n = noun.split(' ')
        scr_max = 0
        temp = []
        for p in permute(n):
            score = 1
            for w in p:
                word = ''.join(w)
                if (word not in counter):
                    counter[word] = naver_search(word)
                score *= counter[word] * (weight ** (len(p)))
                if (verbose == 2):
                    print(word, counter[word])
            if (verbose in [1, 2]):
                print(p, score)
            if (scr_max < score):
                scr_max = score
                temp = p
        for t in temp:
            result.append(''.join(t))
    print(' '.join(result))
    return ' '.join(result)

In [6]:
counter = {}

weight
- 가중치에 따라 점수가 달라짐. 파라미터 조정 필요.

verbose
- 0: 최종 결과 (default)
- 1: 0 + 분할방법에 따른 점수
- 2: 0 + 1 + 각 복합명사 검색 개수

In [7]:
scoring('일중 매매 거래,의미,유의 사항'.split(','), weight=0.15, verbose=2)

일중매매거래 22
[['일중', '매매', '거래']] 3.3
일중 132
매매거래 37
[['일중'], ['매매', '거래']] 2.4725249999999996
일중 132
매매 125
거래 127
[['일중'], ['매매'], ['거래']] 0.08055805957031247
일중매매 20
거래 127
[['일중', '매매'], ['거래']] 1.2858749999999999
의미 131
[['의미']] 19.65
유의사항 72
[['유의', '사항']] 10.799999999999999
유의 99
사항 113
[['유의'], ['사항']] 5.66341875
일중매매거래 의미 유의사항


'일중매매거래 의미 유의사항'

In [8]:
start = datetime.datetime.now()
print(str(start.hour) + ":" + str(start.minute))

15:50


In [9]:
df['pred'] = df['mecab'].apply(lambda x: scoring(x.split(','), weight=0.15))
df

소비자보호실태평가 결과 확인
소비자보호실태평가 평가 방법
소비자보호실태평가 제도
금융감독원 민원대상 금융회사
금융 계좌 조회 사이트
금융용어 등 사이트
정기예금 적금 금리 한눈 비교 곳
이용
자동차 사고발생 당사자 과실비율 산정
교통사고 승합차 렌트 인정 기준
카드단말기 전환 관련
리스 거래 경우 구제
금융회사 민원발생평가 악성민원 판단 기준
자녀 명의 예금 가입 요구 서류
휴면예금
기한 이익상실 연체 기한 이익 상실
채권추심 신용 정보회사 본인 동의 신용정보 신용조회회사 나이스신용평가 코리아크레딧뷰 등 제공 불법
주식 펀드 가입 종합주가지수 상승 손실 발생 점 이해
대포차 자동차보험 계약 체결 차량등록증상 소유주 보험료 할증 등 피해 경우
갱신 실손의료비보험 정기보험 가입 유지 갱신시 보험료인상 이유
청약 당시 보험설계사 고지사항 구두 전달 보험 사 고지 위반 이유 계약 해지
보험 사 보험료 카드결제 문제
보험 회사 등 방문 보험 계약 청약철회
보험 가입
연체 법 절차 진행
카드사 일방 이용한도
카드 발급
계좌 돈 송금
은행 대출 신청 거절 이유
보험계약 후 청약서 보험약관 교부 지금 보험료
금융투자회사 직원 권유 금융투자상품 가입 원금손실 가능 상품 위험 고지 경우 불완전판매 인정
사람 잘못 차량 파손 차량 사용 기간 동안 손해 배상 자동차보험약관 배상 기준
연체 중 이자 납입 변경 가능
사망 자명 예금 입금 인출 제한
기업 대출 등기이사 개인정보제공동의서 요구
증권회사 직원 투자권유 투자 고객 손실 경우 불법행위책임 성립 요건
은행 근저당 설정 아파트 매매거래 주의 사항
동양그룹 분쟁조정 관련
소규모펀드 정의 소규모 펀드투자 유의사항 안내
집합투자기구평가회사 업무 등록 요건
일반사무관리회사 업무 등록 요건
집합투자재산 보관 관리 신탁업자 업무
채권장외거래결제일 중 당일결제 가능여부
상호저축은행 영위 업무 범위
불법채권추심 저축은행 불법 채권추심행위 피해 경우 대응
상호저축은행 대출 이외 대출 취급수수료 대출 취급수수료
개별 상호저축은행 경영공시 곳
상호저

이용 가맹점 상호 주소 신용카드매출전표 기재 것 위장가맹점 신고
연대보증 부탁 후 카드사 신분확인 최근 보증 채무 대신 변제 추심 유선 통화 보증
신용카드 관련 통계
백화점 상품권 신용카드 구입 거부 상품권 카드 구입
신용카드 할부거래 가맹점 대금 부기 입금
상점 일시불 할부거래 거절 불법
카드사 맹점 할부 가능기간 일방 단축
인터넷 쇼핑몰 카드 결제 거부
신용카드 대금 일부 연체 전체 포인트 적립 일부 적립 개선
신용카드 모집인 관련 법규
환대출시 보증인 요구 것 부당
카드
물건 환불 카드 사와 맹점 환불 카드 결제 취소
포인트 적립 이유 일방 차감
카드사 임의 부가서비스 내용 변경 소비자 불리 변경 경우 금감원 제재
카드깡
신용카드현금서비스 이자
신용카드 이용한도 사이 이용한도 감액 전 사전통지 부당
신용카드 모집인 별도 자격요건
길거리 카드 모집 것 위법
카드발급신청 상당 사은품
신용카드 사 금리 제한
체크카드 이용 중 결제 취소 가맹점 환불 일주일 소요 사실
신용카드 부 물건 구입 판매자 연락 거래 취소
물건 분실 카드 판매 카드 회원 카드실물 뒷면서명 대조 가맹점
업종 특수 감안 특정 업종 신용카드 맹점 수수료
신용카드 결제 가능 최저 금액 얼마
공과금 카드 수납
여신전문금융업 법 불법채권추심 규정
카드 연체 일요일 추심업자 전화 불법
카드 연체 밤 추심업자 전화 생활 불법
채권자 가족 카드 이용대금 연체 중 사실 통보 대납 요구 불법
뒷면 서명 카드 분실 카드사 카드 부정 사용금액 보상 사실
전 연회비 카드 해지 연회비 환급
지인 부탁 신용카드 발급 등록 이번달 연회비 인출 환급
타인 신용카드 카드 대금 연체 채권추심
타인 명의 도용 신용카드 발급 카드 이용대금청구
신용카드발급 신청 신용정보제공활용동의서 서명 카드 발급 서명
배우자 분별 신용카드 이용 채무 대신 변제 배우자 신용카드 신규발급
신용카드 배달 분 신용카드 직원 개인정보 노출
동의 아들 가족 카드 발급
소득 신용카드 발급 카드사도 책임 것
소득 신용카드 발급
과거 연체기록 이유 신용 한도

사전 고지 채무 자명 예금 일방 상계처리 가능
급여 압류 후 최저 생활비 연체대금 상계처리 반복
소액 예금 이자
은행 대출금리 것
앞 수표 주민등록번호 기재
대출 금리 상향 조정 대출 갱신 경우
대출 상환방법 사전통보 분할납부 시납 변경 것 가능
대출갱신 구두확약 후 대출갱신 일 래시 영업방침 변경 등 이유 일방 거절 가능
변동금리부 대출 변동 금리 변동 가능 설명 대출 계약 것 가능
적용 금리비교 은행 신용 조회 이유 대출 거부
학자금 출시 부모동의절차 대출 부당 취급
학자금 전세자금대출 요건 완화
담보제공 연대보증 포괄근보증 입보요구 가능
연대보증 위험 보증채무 내용 설명 채 보증계약 체결 경우 유효
보증인 자서날 여부 미확인 상태 대출 취급
포괄근보증 요구 가능
포괄근담보 요구 가능
보험 회사 경영상태 확인 방법
보험 회사 재무현황 등 비교 방법
휴면보험금 조회 방법
보험대리점 등록증 발급 절차
자산운용 회사 투자자 자금 운용
펀드 입시 고려 사항 것
펀드 예금자 보호대상
투자 펀드 수익
펀드 투자 원금 보장
적립식펀드 투자 수익 데 사실
사모투자전문회사 투자자 자격 제한
펀드 환매
펀드 판매 가격 결정
보험 회사 가지급보험금 지급 지연 가지급 보험금 산정 지급 경우
보험 회사 가지급보험금 지급 거절 경우
가지급보험금 지급액
가지급 보험금 청구일 며칠 지급
개인 연금 설계서 최저보증 율 적용
개인연금 보험가입 당시 연금 예상액 실제 연금지급액 상당 차이 이유
연금저축 판매 회사 상품 정보 확인
개인 연금보험 납입기간 중 보험료 증액 가능
연금저축 상품 금융 회사
개인연금 상품 종류
가지급 보험금청구절차
가지급보험금 제도 이용
가지급 보험금 청구
가지급 보험금
해외 도박 관련 가맹점 신용카드결제 가능여부
동종 유사 보험 범위 설명 기재 내용 비교 설명 방법
카드 채무연체 고지 변제 요구 여부
해외선물거래
선물회사 경영건전 확인 방법
영업용 자본비율 증권회사
영업용 자본비율
후 순위 차입금
전체 증권회사 위탁매매 수수료 비교 곳
금융투자업자 임직원 금융투자상

,title,answer,mecab,pred
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,소비자보호실태평가 결과 어디 확인,"소비자 보호 실태 평가 결과,확인",소비자보호실태평가 결과 확인
1,소비자보호실태평가의 평가방법이 궁금합니다.,소비자보호실태평가 평가방법,"소비자 보호 실태 평가,평가 방법",소비자보호실태평가 평가 방법
2,소비자보호실태평가는 어떤 제도인가요?,소비자보호실태평가 제도,"소비자 보호 실태 평가,제도",소비자보호실태평가 제도
3,금융감독원의 민원대상 금융회사가 궁금합니다.,금융감독원 민원대상 금융회사,"금융 감독원,민원 대상 금융 회사",금융감독원 민원대상 금융회사
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,내 금융계좌 번 조회 사이트,"금융 계좌,조회,사이트",금융 계좌 조회 사이트
5,어려운 금융용어 등을 쉽게 찾아볼 수 있는 사이트는 없나요?,금융용어 등 사이트,"금융 용어,등,사이트",금융용어 등 사이트
6,정기예금 및 적금 금리를 한눈에 비교해 볼 수 있는 곳은 없나요?,정기예금 적금금리 한눈 비교 곳,"정기예금,적금 금리,한눈,비교,곳",정기예금 적금 금리 한눈 비교 곳
7,「보이는 ARS」는 어떻게 이용할 수 있나요?,이용,이용,이용
8,자동차사고 발생시 양당사자의 과실비율은 어떻게 산정하나요?,자동차사고 발생 양당사자 과실비율 산정,"자동차 사고 발생,당사자,과실 비율,산정",자동차 사고발생 당사자 과실비율 산정
9,"교통사고 시 SUV, RV, 승합차의 렌트비 인정기준은 어떻게 되나요?",교통사고 승합차 렌트비 인정기준,"교통사고,승합차,렌트,인정 기준",교통사고 승합차 렌트 인정 기준


In [10]:
end = datetime.datetime.now()
print(str(end.hour) + ":" + str(end.minute))

17:2


## 결과

In [17]:
df

,title,answer,mecab,pred
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,소비자보호실태평가 결과 어디 확인,"소비자 보호 실태 평가 결과,확인",소비자보호실태평가 결과 확인
1,소비자보호실태평가의 평가방법이 궁금합니다.,소비자보호실태평가 평가방법,"소비자 보호 실태 평가,평가 방법",소비자보호실태평가 평가 방법
2,소비자보호실태평가는 어떤 제도인가요?,소비자보호실태평가 제도,"소비자 보호 실태 평가,제도",소비자보호실태평가 제도
3,금융감독원의 민원대상 금융회사가 궁금합니다.,금융감독원 민원대상 금융회사,"금융 감독원,민원 대상 금융 회사",금융감독원 민원대상 금융회사
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,내 금융계좌 번 조회 사이트,"금융 계좌,조회,사이트",금융 계좌 조회 사이트
5,어려운 금융용어 등을 쉽게 찾아볼 수 있는 사이트는 없나요?,금융용어 등 사이트,"금융 용어,등,사이트",금융용어 등 사이트
6,정기예금 및 적금 금리를 한눈에 비교해 볼 수 있는 곳은 없나요?,정기예금 적금금리 한눈 비교 곳,"정기예금,적금 금리,한눈,비교,곳",정기예금 적금 금리 한눈 비교 곳
7,「보이는 ARS」는 어떻게 이용할 수 있나요?,이용,이용,이용
8,자동차사고 발생시 양당사자의 과실비율은 어떻게 산정하나요?,자동차사고 발생 양당사자 과실비율 산정,"자동차 사고 발생,당사자,과실 비율,산정",자동차 사고발생 당사자 과실비율 산정
9,"교통사고 시 SUV, RV, 승합차의 렌트비 인정기준은 어떻게 되나요?",교통사고 승합차 렌트비 인정기준,"교통사고,승합차,렌트,인정 기준",교통사고 승합차 렌트 인정 기준


In [16]:
counter

{'일중매매거래': 22,
 '일중': 132,
 '매매거래': 37,
 '매매': 125,
 '거래': 127,
 '일중매매': 20,
 '의미': 131,
 '유의사항': 72,
 '유의': 99,
 '사항': 113,
 '소비자보호실태평가결과': 0,
 '소비자': 123,
 '보호실태평가결과': 0,
 '보호': 117,
 '실태평가결과': 1,
 '실태': 105,
 '평가결과': 16,
 '평가': 109,
 '결과': 142,
 '실태평가': 29,
 '보호실태': 19,
 '보호실태평가': 30,
 '소비자보호': 44,
 '소비자보호실태': 14,
 '소비자보호실태평가': 22,
 '확인': 114,
 '평가방법': 34,
 '방법': 128,
 '제도': 134,
 '금융감독원': 126,
 '금융': 123,
 '감독원': 113,
 '민원대상금융회사': 0,
 '민원': 113,
 '대상금융회사': 0,
 '대상': 64,
 '금융회사': 49,
 '회사': 136,
 '대상금융': 0,
 '민원대상': 10,
 '민원대상금융': 0,
 '금융계좌': 39,
 '계좌': 131,
 '조회': 128,
 '사이트': 101,
 '금융용어': 44,
 '용어': 103,
 '등': 146,
 '정기예금': 88,
 '적금금리': 2,
 '적금': 124,
 '금리': 163,
 '한눈': 75,
 '비교': 109,
 '곳': 112,
 '이용': 108,
 '자동차사고발생': 0,
 '자동차': 114,
 '사고발생': 21,
 '사고': 135,
 '발생': 120,
 '자동차사고': 17,
 '당사자': 90,
 '과실비율': 76,
 '과실': 145,
 '비율': 104,
 '산정': 123,
 '교통사고': 76,
 '승합차': 93,
 '렌트': 183,
 '인정기준': 32,
 '인정': 110,
 '기준': 135,
 '카드단말기': 94,
 '카드': 149,
 '단말기': 90,
 '전환': 124,
 '관련': 147,


## 성능 평가

In [11]:
def evaluating(x):
    answer, pred = x
    count = 0
    for w in answer.split(' '):
        if w in pred.split(' '):
            count += 1
    return count / len(answer.split(' '))

In [13]:
df['eval'] = df[['answer', 'pred']].apply(lambda x: evaluating(x), axis=1)
df

,title,answer,mecab,pred,eval
0,소비자보호실태평가 결과는 어디서 확인할 수 있나요?,소비자보호실태평가 결과 어디 확인,"소비자 보호 실태 평가 결과,확인",소비자보호실태평가 결과 확인,0.750000
1,소비자보호실태평가의 평가방법이 궁금합니다.,소비자보호실태평가 평가방법,"소비자 보호 실태 평가,평가 방법",소비자보호실태평가 평가 방법,0.500000
2,소비자보호실태평가는 어떤 제도인가요?,소비자보호실태평가 제도,"소비자 보호 실태 평가,제도",소비자보호실태평가 제도,1.000000
3,금융감독원의 민원대상 금융회사가 궁금합니다.,금융감독원 민원대상 금융회사,"금융 감독원,민원 대상 금융 회사",금융감독원 민원대상 금융회사,1.000000
4,내 금융계좌를 한번에 조회할 수 있는 사이트는 없나요?,내 금융계좌 번 조회 사이트,"금융 계좌,조회,사이트",금융 계좌 조회 사이트,0.400000
5,어려운 금융용어 등을 쉽게 찾아볼 수 있는 사이트는 없나요?,금융용어 등 사이트,"금융 용어,등,사이트",금융용어 등 사이트,1.000000
6,정기예금 및 적금 금리를 한눈에 비교해 볼 수 있는 곳은 없나요?,정기예금 적금금리 한눈 비교 곳,"정기예금,적금 금리,한눈,비교,곳",정기예금 적금 금리 한눈 비교 곳,0.800000
7,「보이는 ARS」는 어떻게 이용할 수 있나요?,이용,이용,이용,1.000000
8,자동차사고 발생시 양당사자의 과실비율은 어떻게 산정하나요?,자동차사고 발생 양당사자 과실비율 산정,"자동차 사고 발생,당사자,과실 비율,산정",자동차 사고발생 당사자 과실비율 산정,0.400000
9,"교통사고 시 SUV, RV, 승합차의 렌트비 인정기준은 어떻게 되나요?",교통사고 승합차 렌트비 인정기준,"교통사고,승합차,렌트,인정 기준",교통사고 승합차 렌트 인정 기준,0.500000


In [14]:
acc = (df['eval'].sum() / len(df)) * 100
acc

68.83269501278696

## 결과 저장

In [15]:
title = 'searched_{0}.csv'.format(round(acc, 2))
df.to_csv(title)